# Computer Vision Systems
## Traditional Vision Demo
In this notebook we demonstrate the usage of simple computer vision algorithms using OpenCV and Python

### Installing OpenCV

In [0]:
!pip install opencv-python

### Downloading images

In [0]:
!rm *.png
!rm *.jpg
!rm *.avi
!rm *.zip
!wget http://3dmr.iit.bme.hu/edu/DL/Downloads.zip
!unzip Downloads.zip

## Image basics
### Read and dislay

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

img = cv2.imread("Lena.png")
plt.figure(figsize=(6,6))
plt.imshow(img)

### Color space conversions

In [0]:
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(img_rgb)
plt.subplot(1,2,2)
plt.imshow(img_gray,cmap='gray')

### Thresholding

In [0]:
_,img_bin = cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)
_,img_bin2 = cv2.threshold(img_gray,50,255,cv2.THRESH_BINARY)
_,img_bin3 = cv2.threshold(img_gray,200,255,cv2.THRESH_BINARY)
plt.figure(figsize=(20,20))
plt.subplot(1,3,1)
plt.imshow(img_bin,cmap='gray')
plt.subplot(1,3,2)
plt.imshow(img_bin2,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(img_bin3,cmap='gray')

### Histogram and equalization

In [0]:
plt.figure(figsize=(25,10))
hist = cv2.calcHist([img_gray],[0],None,[256],[0,256])
img_eq = cv2.equalizeHist(img_gray)
hist2 = cv2.calcHist([img_eq],[0],None,[256],[0,256])
plt.subplot(2,2,1)
plt.imshow(img_gray,cmap='gray')
plt.subplot(2,2,2)
plt.bar(np.arange(256),hist[:,0])
plt.subplot(2,2,3)
plt.imshow(img_eq,cmap='gray')
plt.subplot(2,2,4)
plt.bar(np.arange(256),hist2[:,0])

### Filtering
#### Add noise

In [0]:
def noisy(noise_typ,image):
   if noise_typ == "gauss":
      row,col= image.shape
      mean = 0
      sigma = 15
      gauss = np.random.normal(mean,sigma,(row,col))
      gauss = gauss.reshape(row,col)
      noisy = image + gauss
      return noisy
   elif noise_typ == "snp":
      row,col = image.shape
      s_vs_p = 0.5
      amount = 0.02
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 255

      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out

#### Create images

In [0]:
img_gauss = noisy("gauss",img_gray)
img_snp = noisy("snp",img_gray)
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(img_gauss,cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_snp,cmap='gray')

#### Filter Images

In [0]:
img_gfilt = cv2.GaussianBlur(img_gauss,(3,3),0)
img_mfilt = cv2.medianBlur(img_snp,3)
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(img_gfilt,cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_mfilt,cmap='gray')

## Fourier Transform

In [0]:
f = np.fft.fft2(img_gray)
fshift = np.fft.fftshift(f)
fshift_filter = fshift.copy()
fshift_filter2 = fshift.copy()
center = fshift_filter.shape[0] // 2, fshift_filter.shape[1] // 2
fshift_filter[center[0]-20:center[0]+20,center[1]-20:center[1]+20] = 0
fshift_filter2[0:center[0]-20,:] = 0
fshift_filter2[center[0]+20:,:] = 0
fshift_filter2[:,0:center[1]-20] = 0
fshift_filter2[:,center[1]+20:] = 0
f_filter = np.fft.ifftshift(fshift_filter)
img_filter = np.abs(np.fft.ifft2(f_filter))
f_filter2 = np.fft.ifftshift(fshift_filter2)
img_filter2 = np.abs(np.fft.ifft2(f_filter2))

magnitude_spectrum = 20*np.log(np.abs(fshift))
magnitude_spectrum_filt = 20*np.log(np.abs(fshift_filter)+1)
magnitude_spectrum_filt2 = 20*np.log(np.abs(fshift_filter2)+1)

plt.figure(figsize=(25,15))
plt.subplot(2,3,1)
plt.imshow(img_gray,cmap='gray')
plt.subplot(2,3,2)
plt.imshow(img_filter,cmap='gray')
plt.subplot(2,3,3)
plt.imshow(img_filter2,cmap='gray')
plt.subplot(2,3,4)
plt.imshow(magnitude_spectrum,cmap='gray')
plt.subplot(2,3,5)
plt.imshow(magnitude_spectrum_filt,cmap='gray')
plt.subplot(2,3,6)
plt.imshow(magnitude_spectrum_filt2,cmap='gray')

## Feature detection

### Edges

#### Sobel and Laplace

In [0]:
featImg = cv2.imread("sudoku-original.jpg",0)

laplacian = np.absolute(cv2.Laplacian(featImg,cv2.CV_64F))
sobelx = np.absolute(cv2.Sobel(featImg,cv2.CV_64F,1,0,ksize=5))
sobely = np.absolute(cv2.Sobel(featImg,cv2.CV_64F,0,1,ksize=5))

plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
plt.imshow(featImg,cmap='gray')
plt.subplot(2,2,2)
plt.imshow(laplacian,cmap='gray')
plt.subplot(2,2,3)
plt.imshow(sobelx,cmap='gray')
plt.subplot(2,2,4)
plt.imshow(sobely,cmap='gray')

### Canny and Hough



In [0]:
edges = cv2.Canny(featImg,50,120)

lines = cv2.HoughLines(edges,1,np.pi/180,150)

lineIm = cv2.cvtColor(featImg,cv2.COLOR_GRAY2BGR)

for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(lineIm,(x1,y1),(x2,y2),(255,0,0),2)

plt.figure(figsize=(25,25))
plt.subplot(1,3,1)
plt.imshow(featImg,cmap='gray')
plt.subplot(1,3,2)
plt.imshow(edges,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(lineIm,cmap='gray')

### Template Matching

In [0]:
mario_rgb = cv2.imread('mario.jpg')
mario_gray = cv2.cvtColor(mario_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('mario_template.jpg',0)
w, h = template.shape[::-1]
   
res = cv2.matchTemplate(mario_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc = np.where( res >= threshold)
for pt in zip(*loc[::-1]):
    cv2.rectangle(mario_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
mario_rgb = cv2.cvtColor(mario_rgb, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,10))
plt.imshow(mario_rgb)

### ORB Keypoint Matching

In [0]:
img1 = cv2.imread('box.jpg',0)          # queryImage
img2 = cv2.imread('box_in_scene.png',0) # trainImage
    
# Initiate ORB detector
orb = cv2.ORB_create()
    
# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
    
# cv2.drawMatchesKnn expects list of lists as matches.
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=2)

plt.figure(figsize=(20,20))
plt.imshow(img3)

## Binary Images

### Erosion, dilation

In [0]:
j_img = cv2.imread('j.png',0)

kernel = np.ones((5,5),np.uint8)

erosion = cv2.erode(j_img,kernel,iterations = 1)
dilation = cv2.dilate(j_img,kernel,iterations = 1)

plt.figure(figsize=(15,15))
plt.subplot(1,3,1)
plt.imshow(j_img,cmap='gray')
plt.subplot(1,3,2)
plt.imshow(erosion,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(dilation,cmap='gray')

### Opening, Closing

In [0]:
opening = cv2.imread('opening.png',0)
closing = cv2.imread('closing.png',0)

opened = cv2.morphologyEx(opening, cv2.MORPH_OPEN, kernel)
closed = cv2.morphologyEx(closing, cv2.MORPH_CLOSE, kernel)

plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.imshow(opening,cmap='gray')
plt.subplot(2,2,2)
plt.imshow(opened,cmap='gray')
plt.subplot(2,2,3)
plt.imshow(closing,cmap='gray')
plt.subplot(2,2,4)
plt.imshow(closed,cmap='gray')

## Coin Segmentation using marker watershed

### Threshold

In [0]:
coin_img = cv2.imread('water_coins.jpg')
coin_gray = cv2.cvtColor(coin_img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(coin_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

plt.figure(figsize=(7,7))
plt.imshow(thresh,cmap='gray')

### Eroision to extract sure foreground area

In [0]:
sure_fg = cv2.erode(thresh,kernel,iterations=7)

# sure background area
sure_bg = cv2.dilate(thresh,kernel,iterations=2)
unknown = cv2.subtract(sure_bg,sure_fg)

plt.figure(figsize=(7,7))
plt.imshow(sure_fg,cmap='gray')

### Connected components to label foreground pieces

In [0]:
 # Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
     
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
     
# Now, mark the region of unknown with zero
markers[unknown==255] = 0
plt.figure(figsize=(7,7))
plt.imshow(markers)

### Run Watershed

In [0]:
markers = cv2.watershed(coin_img,markers)
coin_img[markers == -1] = [255,0,0]

plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(markers)
plt.subplot(1,2,2)
plt.imshow(coin_img)

## Video Analytics
### Optical Flow

In [0]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


cap = cv2.VideoCapture("vtest.avi")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
     
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
frame_height,frame_width = prvs.shape
out = cv2.VideoWriter('optflow.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width*2,frame_height))

cntr = 0
bar = display(progress(cntr, length), display_id=True)
     
while(1):
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    
    frame = np.concatenate((frame2,bgr),1)
    out.write(frame)
    
    prvs = next
    cntr += 1
    bar.update(progress(cntr, length))

cap.release()
out.release()

### Background Subtraction

In [0]:
cap = cv2.VideoCapture("vtest.avi")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
     
ret, frame1 = cap.read()

frame_height,frame_width = prvs.shape
out = cv2.VideoWriter('bg.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width*2,frame_height))

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

cntr = 0
bar = display(progress(cntr, length), display_id=True)
     
while(1):
    ret, frame2 = cap.read()
    if not ret:
        break
        
    fgmask = fgbg.apply(frame2)
    bgr = cv2.cvtColor(fgmask,cv2.COLOR_GRAY2BGR)
    
    frame = np.concatenate((frame2,bgr),1)
    out.write(frame)
    
    cntr += 1
    bar.update(progress(cntr, length))

cap.release()
out.release()

## Download video files
To download the created videos, click the ![alt text](http://3dmr.iit.bme.hu/edu/DL/icon.png) icon on the top left, then select the files tab. From there, right click **optflow.mp4** and **bg.mp4** to download them to your computer.